In [3]:
import requests


In [4]:
s = requests.Session()

In [7]:
baseAddress = "https://learning.whchem.com:6443/"
defaultHeader = {"User-Agent": "Mozilla /5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1"}
user = {"username": "00024849", "password": "ytpu2021"}


In [11]:
response = s.post(baseAddress+"Api/User/Login", headers=defaultHeader, data=user)
print(response.text)
userStat = s.get(baseAddress+"Api/Common/Task/GetMyTaskInfo", headers=defaultHeader)
print(userStat.text)

{"state":"success","msg":"ok","data":{"LoginId":"bbc9fbf9-c098-4fdc-8490-e4ae6a74f0cb","LoginYear":2022,"TokenID":"7035adde-5a61-4e48-a533-f7a04e6e6618","AppMode":1,"CustomerToken":"allRanking","CustomerId":1,"CustomerRootCompanyID":"0","Sex":"男","Telephone":"19906049273","PhoneCountryPrefix":"+86","Email":"HZPAN@WHCHEM.COM","Ip":2084994367,"IsSelectDevice":false,"Photo":"https://filelearning.whchem.com/Download.aspx?id=685ddd92-e308-4187-babd-9add373cb1ea&user=test&company=0&verify=91E3C92FB4691937E5B10BE7D60203F03E0F95F700C79F2A9597BB29183ED0730979F551A7B1193D7333F4E8F537932DE43DE0B83E73AD8F6A7345372E8AE3F2A82B22FAC6B3391B1427570D602F84AB7FEEB6815253BD74F7B5021F368AE2A79F99568DAB1DDA46C807CC3BC3D58F0380756AD744B9ED9D807FED5E40328D74&isThumbnail=1","OfficalPhoto":"","isActiveTag":true,"UserTag":null,"Username":"00024849","Realname":"潘昊泽","CompanyId":"0:21:6:3","CompanyName":"光化"}}
{"state":"nologin","msg":"未登录！","data":null}
